In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import time
import argparse
#import imutils
#from imutils.video import VideoStream
#from imutils.video import FPS
import IPython


# Picturemode or not
picturemode = False

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

In [3]:
# CALIBRATE CAMERA
    
list_objp = []
list_imgp = []
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

# List of calibration images
list_images = os.listdir("camera_cal/")

obj_points = np.zeros((nx*ny, 3), np.float32)
obj_points[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Loop through every image for calibration
for img_name in list_images:
    chess = mpimg.imread("camera_cal/" + img_name)
    gray = cv2.cvtColor(chess, cv2.COLOR_RGB2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    if ret == True:
        if(picturemode == True):
            chess = cv2.drawChessboardCorners(chess, (nx,ny), corners, ret)
            plt.imshow(chess)
        #mpimg.imsave(fname = "output_images/calibration/"+img_name, arr=chess)
        list_objp.append(obj_points)
        list_imgp.append(corners)
    
# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(list_objp, list_imgp, gray.shape[::-1], None, None)

In [4]:
#Undistort camera example


if(picturemode == True):
    img_name = 'calibration1.jpg'
    chess = mpimg.imread('camera_cal/'+img_name)
    undist = cv2.undistort(chess, mtx, dist, None, mtx)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    ax1.set_title('Original Image', fontsize=50)
    ax1.imshow(chess)
    ax2.set_title('Undistorted Image', fontsize=50)
    ax2.imshow(undist)
    plt.show()
    
    img_name = 'straight_lines1.jpg'
    chess = mpimg.imread('test_images/'+img_name)
    undist = cv2.undistort(chess, mtx, dist, None, mtx)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    #plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    ax1.set_title('Original Image', fontsize=50)
    ax1.imshow(chess)
    ax2.set_title('Undistorted Image', fontsize=50)
    ax2.imshow(undist)
    plt.show()

#FINISH!!

In [5]:
# Convert to HLS color space and separate the S channel
def findLanes(img):
    
    # !!!!! in case of single image, activate that!
    #img_name = 'straight_lines2.jpg'
    #img = mpimg.imread('test_images/'+img_name)

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, 7) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 80
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    if (picturemode == True):
        # Plotting thresholded images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Stacked thresholds')
        ax1.imshow(color_binary)
        ax2.set_title('Combined S channel and gradient thresholds')
        ax2.imshow(combined_binary, cmap='gray')
        plt.show()
    
    return combined_binary


In [6]:
# check for both straight test images
#for img_name in ["straight_lines1.jpg", "straight_lines2.jpg"]:
#img = mpimg.imread('test_images/' + img_name)
#img = cv2.undistort(img, mtx, dist, None, mtx)

def warpImage(img, combined_binary):

    dx = 300
    dy = 0

    # use undistorted image
    x_shape = img.shape[1]
    y_shape = img.shape[0]

    # transformation
    #source_points = np.float32([[x_shape//2-75, 450],[x_shape//2+75, 450],[x_shape-dx, y_shape], [dx, y_shape]])
    source_points = np.float32([[x_shape//2-38, 445],[x_shape//2+38, 445],[x_shape-195, y_shape], [195, y_shape]])
    destination_points = np.float32([[dx,dy],[x_shape-dx,dy],[x_shape-dx,y_shape-dy],[dx,y_shape-dy]])

    M = cv2.getPerspectiveTransform(source_points, destination_points)
    Minv = cv2.getPerspectiveTransform(destination_points, source_points)

    warped = cv2.warpPerspective(img, M, (x_shape, y_shape), flags=cv2.INTER_LINEAR)

    # draw sourve and destination points into image
    pts_src = source_points.astype(int).reshape((-1,1,2))
    pts_dst = destination_points.astype(int).reshape((-1,1,2))

    if (picturemode == True):
        fig = plt.figure(figsize=(20,10))
        # source
        fig.add_subplot(1,2,1)
        plt.title('Original Undistorted Image');
        plt.plot(source_points[:,0], source_points[:,1],'ro')
        plt.imshow(cv2.polylines(img,[pts_src],True,(255,0,0)))
        # transformed
        fig.add_subplot(1,2,2)
        plt.title('Warped Image');
        plt.plot(destination_points[:,0], destination_points[:,1],'ro')
        plt.imshow(cv2.polylines(warped,[pts_dst],True,(255,0,0)))
        plt.show()

    binary_warped = cv2.warpPerspective(combined_binary, M, (x_shape, y_shape), flags=cv2.INTER_LINEAR)

    if (picturemode == True):
        fig = plt.figure(figsize=(20,10))
        ## source
        fig.add_subplot(1,2,1)
        cv2.polylines(combined_binary,[pts_src],True,(0,0,100))
        plt.title('Original Undistorted Image');
        plt.plot(source_points[:,0], source_points[:,1],'ro')
        plt.imshow(combined_binary, cmap='gray')
        # transformed
        fig.add_subplot(1,2,2)
        plt.title('Warped Image');
        plt.plot(destination_points[:,0], destination_points[:,1],'ro')
        plt.imshow(cv2.polylines(binary_warped,[pts_dst],True,(0,0,100)))
        plt.show()
    
    return binary_warped, Minv

In [7]:
#Finding the Lines: Histogram Peaks                  ##### DEPRECATED!!!
#binary_warped is the next step picture

def getHistogram(binary_warped):
    
    if (picturemode == True):
        bottom_half = binary_warped[binary_warped.shape[0]//2:,:]
        histogram = np.sum(bottom_half, axis=0)
        plt.plot(histogram)
        plt.show
    
    return histogram


In [8]:
def lane_sanity_check(left_fitx, right_fitx, left_line, right_line):
    flag = True

    ld_bottom = right_fitx[719] - left_fitx[719]
    ld_mid = right_fitx[320] - left_fitx[320]
    ld_top = right_fitx[0] - left_fitx[0]
    d_radius = np.absolute(left_line.radius_of_curvature / right_line.radius_of_curvature)
    

    if(0.5 < d_radius > 1.5):
        flag = False
        #print('Wrong Radius!!!!!!!' + str(d_radius))
    if ((ld_bottom < 600) or (ld_bottom > 800)): 
        flag = False
    if ((ld_mid < 600) or (ld_mid > 800)): 
        flag = False
    if ((ld_top < 600) or (ld_top > 800)): 
        flag = False
    
        # In case everything went good...
    if (flag == True):
        # save recent fitted x-values
        left_line.detected = True
        right_line.detected = True
        left_line.best_fit = np.mean(np.array(left_line.recent_xfitted[-20:]),axis=0) 
        right_line.best_fit = np.mean(np.array(right_line.recent_xfitted[-20:]),axis=0)
        # save x-value
        left_line.recent_xfitted.append(left_fitx[-1])
        right_line.recent_xfitted.append(right_fitx[-1])
    else:
        #print('Flag is FALSE!!!!!!!!!!')
        left_line.detected = False 
        right_line.detected = False
        #left_line.current_fit = left_line.best_fit
        #right_line.current_fit = right_line.best_fit
        
    return flag

In [9]:
def findingLines(binary_warped, left_line, right_line):
     
    # START WITH THE POLY APPROACH
    if left_line.detected and right_line.detected:
        leftx, lefty, rightx, righty = search_around_poly(binary_warped, left_line, right_line)
        #print('Using Poly')
    
    # IF THAT FAILS, USE SLIDING WINDOWS AGAIN AND START WITH A GOOD BASELINE
    if (left_line.detected == False) or (right_line.detected == False):
        leftx, lefty, rightx, righty = sliding_window(binary_warped, left_line, right_line)
        #print('Using Sliding')
    
    return leftx, lefty, rightx, righty

In [10]:
#Finding the Lines: Sliding Window

# Picture Sliding Boxes/Windows yes or no
slidingwindow = False

def sliding_window(binary_warped, left_line, right_line):

    #def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    #histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0) #!!!! OLD!
    
    if(slidingwindow == True):
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
        
    if len(left_line.recent_xfitted) > 20 and len(right_line.recent_xfitted) > 20:
        # start with average of last start points
        #left_line.bestx = np.average(left_line.recent_xfitted[-20:-1])
        #right_line.bestx = np.average(right_line.recent_xfitted[-20:-1])
        left_line.bestx = np.average(left_line.recent_xfitted)
        right_line.bestx = np.average(right_line.recent_xfitted)
        #print('startbase')
    else: 
        #Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        #print('use histogram')
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        left_line.bestx = np.argmax(histogram[:midpoint])
        right_line.bestx = np.argmax(histogram[midpoint:]) + midpoint
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = left_line.bestx
    rightx_current = right_line.bestx

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        if (slidingwindow == True):
            # Draw the windows on the visualization image -> comment this out!
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        print('ERROR in concatenate')

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if (picturemode == True):
        print('Pixels of:', 'LeftX', leftx, 'LeftY', lefty, 'RightX', rightx, 'RightY', righty)
        
    return leftx, lefty, rightx, righty


In [11]:
def search_around_poly(binary_warped, left_line, right_line):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Search area based on activated x-values within the +/- margin of previous polynomial 
    left_lane_inds = (nonzerox > left_line.current_fit[0]*(nonzeroy)**2 + left_line.current_fit[1]*nonzeroy + left_line.current_fit[2] - 100) & \
                     (nonzerox < left_line.current_fit[0]*(nonzeroy)**2 + left_line.current_fit[1]*nonzeroy + left_line.current_fit[2] + 100)
    right_lane_inds = (nonzerox > right_line.current_fit[0]*(nonzeroy)**2 + right_line.current_fit[1]*nonzeroy + right_line.current_fit[2] - 100) & \
                      (nonzerox < right_line.current_fit[0]*(nonzeroy)**2 + right_line.current_fit[1]*nonzeroy + right_line.current_fit[2] + 100)

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    return leftx, lefty, rightx, righty

In [12]:
def fit_polynomial_old(binary_warped, leftx, lefty, rightx, righty):                       ##### DEPRECATED!!!
    # Find our lane pixels first
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty    
    
    # save x-value
    left_line.recent_xfitted.append(left_fitx[-1])
    right_line.recent_xfitted.append(right_fitx[-1])

    
    if (picturemode == True):
        ## Visualization ##
        
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Plots the left and right polynomials on the lane lines
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.title('Fit Polynomial')
        plt.imshow(out_img)
        plt.show()
     
    return left_fitx, right_fitx, left_fit, right_fit

In [13]:
#Calculate the radius - Lesson 7                  ##### DEPRECATED!!!
def calculateRadius(left_fitx, left_fit, right_fit):
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(left_fitx)
    
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    if (picturemode == True):
        print('Left-Curve-Radius:', left_curverad, '   Right-Curve-Radius:', right_curverad)
        
    return left_curverad, right_curverad

In [14]:
def car_off(img, leftx, rightx):                       ##### DEPRECATED!!!
    xm_per_pix = 3.7/img.shape[1] # meters per pixel in x dimension
    mid_imgx = img.shape[1]//2
    ## Car position in lane
    car_pos = (leftx[-1] + rightx[-1])/2
    #car offset 
    off = (mid_imgx - car_pos) * xm_per_pix
    return off

In [15]:
#From Pixels to Real-World                  ##### DEPRECATED!!!

def calculateRealRadius(img, leftx, lefty, rightx, righty, left_fit):
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(left_fit)  #y_eval = np.max(left_fitx)  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    #leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    #rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    
    #Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/img.shape[0] # meters per pixel in y dimension
    xm_per_pix = 3.7/img.shape[1] # meters per pixel in x dimension

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculation of R_curve (radius of curvature)
    real_left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    real_right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
    if (picturemode == True):
        print('Left-Curve-Radius:', real_left_curverad, 'm', '   Left-Curve-Radius:', real_right_curverad, 'm')

    return real_left_curverad, real_right_curverad

In [16]:
def fit_polynomial(img, binary_warped, leftx, lefty, rightx, righty, Minv, left_line, right_line):
    # Find our lane pixels first
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    left_line.current_fit = left_fit
    right_line.current_fit = right_fit
    
    try:
        left_fitx = left_line.current_fit[0]*ploty**2 + left_line.current_fit[1]*ploty + left_line.current_fit[2]
        right_fitx = right_line.current_fit[0]*ploty**2 + right_line.current_fit[1]*ploty + right_line.current_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(left_line.current_fit)  #y_eval = np.max(left_fitx)  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    #leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    #rightx = rightx[::-1]  # Reverse to match top-to-bottom in y

    #Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/img.shape[0] # meters per pixel in y dimension
    xm_per_pix = 3.7/img.shape[1] # meters per pixel in x dimension

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculation of R_curve (radius of curvature)               ## Abweichung !!!!
    left_line.radius_of_curvature = ((1 + (2*left_line.current_fit[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_line.radius_of_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])


    # curvature
    y_eval_p = np.max(ploty)
    y_eval_m = np.max(ploty)*ym_per_pix
    left_line.radius_of_curvature = (1+(2*left_line.current_fit[0]*xm_per_pix/(ym_per_pix**2)*y_eval_m + xm_per_pix/ym_per_pix*left_line.current_fit[1])**2)**(3/2) / np.absolute(2*left_line.current_fit[0]*xm_per_pix/(ym_per_pix**2)) 
    right_line.radius_of_curvature = (1+(2*right_line.current_fit[0]*xm_per_pix/(ym_per_pix**2)*y_eval_m + xm_per_pix/ym_per_pix*right_line.current_fit[1])**2)**(3/2) / np.absolute(2*right_line.current_fit[0]*xm_per_pix/(ym_per_pix**2))

      
    # SANITY CHECKS
    flag = lane_sanity_check(left_fitx, right_fitx, left_line, right_line)
    
        
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Generate x and y values for plotting
    #ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)

    x_shape = img.shape[1]
    y_shape = img.shape[0]

    ## Getting Value for the Center of the Car 
    y_eval_p = np.max(ploty)
    xm_per_pix = 3.7/img.shape[1] # meters per pixel in x dimension
    mid_imgx = img.shape[1]//2
    ## Car position in lane
    car_pos = (leftx[-1] + rightx[-1])/2
    #car offset in meter
    off = (mid_imgx - car_pos) * xm_per_pix
    # distance car to lane position
    #leftx_eval = left_line.current_fit[0]*y_eval**2 + left_line.current_fit[1]*y_eval + left_line.current_fit[2]
    #rightx_eval = right_line.current_fit[0]*y_eval**2 + right_line.current_fit[1]*y_eval + right_line.current_fit[2]
    #left_line.dist_line_car = abs(binary_warped.shape[1]/2 - leftx_eval) * xm_per_pix
    #right_line.dist_line_car = abs(binary_warped.shape[1]/2 - rightx_eval) * xm_per_pix
    #print('leftx[-1]:' + str(leftx[-1]*xm_per_pix) + '  rightx[-1]:' + str(rightx[-1]*xm_per_pix))
    #print('left_line.dist_line_car: ' + str(left_line.dist_line_car) + '  right_line.dist_line_car: ' + str(right_line.dist_line_car))
    
    text = "Radius Left: " + "{:.9f}".format(left_line.radius_of_curvature) + " Radius Right: " + "{:.9f}".format(right_line.radius_of_curvature)
    cv2.putText(result, text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    text = "Car away from lane center: " + "{:.9f}".format(off)
    cv2.putText(result, text, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    #cv2.putText(result, fit_fail_counter, (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    

    if (picturemode == True):
        ## Visualization ##
        
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Plots the left and right polynomials on the lane lines
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.title('Fit Polynomial')
        plt.imshow(out_img)
        plt.show()
        
        
    if (picturemode == True):
        plt.imshow(result)
        plt.title('Resulting Picture')
        plt.show()

    return result

In [17]:
#Copied from Project description                       ##### DEPRECATED!!!

def createImage(img, binary_warped, left_fitx, right_fitx, Minv, real_left_curverad, real_right_curverad, off):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    x_shape = img.shape[1]
    y_shape = img.shape[0]
    text = "Radius Left: " + "{:.9f}".format(real_left_curverad) + " Radius Right: " + "{:.9f}".format(real_right_curverad)
    cv2.putText(result, text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    text = "Car away from lane center: " + "{:.9f}".format(off)
    cv2.putText(result, text, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    #cv2.putText(result, fit_fail_counter, (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    
    if (picturemode == True):
        plt.imshow(result)
        plt.title('Resulting Picture')
        plt.show()

    return result

In [18]:
#My Pipeline starts here

def pipeline(img):
    img = cv2.undistort(img, mtx, dist, None, mtx)    
    combined_binary = findLanes(img)
    binary_warped, Minv = warpImage(img, combined_binary)    
    leftx, lefty, rightx, righty = findingLines(binary_warped, left_line, right_line)
    #left_fitx, right_fitx, left_fit, right_fit = fit_polynomial(binary_warped, leftx, lefty, rightx, righty)
    #left_curverad, right_curverad = calculateRadius(left_fitx, left_fit, right_fitx)
    #real_left_curverad, real_right_curverad = calculateRealRadius(img, leftx, lefty, rightx, righty, left_fit)
    #off = car_off(img, leftx, rightx)
    #result = createImage(img, binary_warped, left_fitx, right_fitx, Minv, real_left_curverad, real_right_curverad, off)
    #if (picturemode == True):
        #getHistogram(binary_warped)
    
    result = fit_polynomial(img, binary_warped, leftx, lefty, rightx, righty, Minv, left_line, right_line)

    return result

In [19]:
left_line = Line()
right_line = Line()

if(picturemode != True):
    yellow_output = 'out_project_video.mp4'
    ##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
    clip = VideoFileClip('project_video.mp4')#.subclip(37,44)
    yellow_clip = clip.fl_image(pipeline)
    %time yellow_clip.write_videofile(yellow_output, audio=False)
else:
    img_name = 'straight_lines1.jpg'
    img = mpimg.imread('test_images/'+img_name)
    pipeline(img)

[MoviePy] >>>> Building video out_project_video.mp4
[MoviePy] Writing video out_project_video.mp4


100%|█████████▉| 1260/1261 [04:45<00:00,  4.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: out_project_video.mp4 

CPU times: user 2min 18s, sys: 17.8 s, total: 2min 36s
Wall time: 4min 48s


In [20]:
if(picturemode != True):
    HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(yellow_output))